<a href="https://colab.research.google.com/github/MojtabaValizadeh/ltl-learning-on-gpus/blob/main/LTL_Learning_on_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <strong>LTL Learning on GPUs</strong>

This notebook contains the code and other artifacts for the  paper

> **LTL Learning on GPUs** \\
> **CAV 2024, 36th International Conference on Computer Aided Verification**

by `Mojtaba Valizadeh`, `Nathanaël Fijalkow` and `Martin Berger`.

A draft of the paper is available at https://arxiv.org/abs/2402.12373.

# <strong>Google Colab's GPUs</strong>

To optimize memory and performance, we recommend upgrading your Colab to `Pro` or `Pro+` versions. These paid versions offer larger memory limits, faster processing speeds, and advanced hardware acceleration, enabling users to execute complex operations with greater efficiency. By contrast, the `free version` of Colab may be subject to limitations that can impact the performance and accuracy of the following tasks.

**To upgrade to Colab Pro:**

- Find the `Colab Pro` tab in `Tools > Settings`.
- Choose your desired plan: `Colab Pro` or `Colab Pro+`.
- Follow the prompts to enter your billing information and complete the purchase process.

Colab Pro is billed on a monthly basis, and you can cancel at any time. Additionally, some countries or regions may not be eligible to purchase Colab Pro at this time, so you may need to check if it is available in your area before proceeding.

**Note:** Even with the free version of Colab, you should be able to connect to certain types of GPUs, though there are some limitations. Everything you need is already set up in this notebook. Now, please run the following script to check if you are connected to a GPU. To run a cell in Colab, you can either click on the "play" button located on the left side of the cell, or you can press "Shift+Enter" on your keyboard while the cell is selected.

In [ ]:
# Check if connected to GPU
gpu_info = ! nvidia-smi
gpu_info = '\n'.join(gpu_info)
if 'failed' in gpu_info or 'not found' in gpu_info:
  print('GPU not found!')
  print("At 'Runtime > Change runtime type', please choose `GPU` for `Hardware accelerator` and try again.")
else:
    ! nvidia-smi
    print()
    print("Great! You are connected to a GPU! Please go on ...")

# <strong>Initialization</strong>

To begin, please run the code below to transfer all of the necessary requirements, including the codes, dependencies, benchmarks, etc, to this notebook.

In [ ]:
import os
if not os.path.isdir("/content/ltl-learning-on-gpus"):
    !git clone https://github.com/MojtabaValizadeh/ltl-learning-on-gpus
    print("Done")
else:
    print("The repository already exists in the notebook!")

# <strong>Compile the GPU code</strong>

To compile and run the GPU code, please run the following scripts.

In [ ]:
print("Compiling the GPU code...")
cu_path  = "/content/ltl-learning-on-gpus/code/ltli6463.cu"
lib_path = "/content/ltl-learning-on-gpus/code/modified_libraries"

! nvcc --extended-lambda -D MEASUREMENT_MODE -I {lib_path} {cu_path} -o ltli6463

print("Done")

**Note** You can remove "-D MEASUREMENT_MODE" from the compilation instructions above to see more logs and information while running the code.

# <strong>Run the GPU code</strong>

In order to run any small examples (with fewer than 64 traces with max length of 63 in the positive and negative traces in the input file), you can use the GPU code directly by executing
```
! ./ltli6463 <input_path> <c1> <c2> <c3> <c4> <c5> <c6> <c7> <c8> <maxCost> <RlxUnqChkType> <negType>
```
where
1. `input_path` refers to the address of the input file that contains your positive and negative traces.
2. (`c1`, `c2`, `c3`, `c4`, `c5`, `c6`, `c7`, `c8`) are 8 small positive integers for the costs of (`a`, `~`, `&`, `|`, `X`, `F`, `G`, `U`).
3. `maxCost` parameter is an integer that sets an upper limit on the cost of the regular expression that the algorithm will search for. In most cases, you can use a reasonably large integer, such as 500, which is appropriate for our cost functions.
4. `RlxUnqChkType` is an integer in {`1`, `2`, `3`} for choosing one of the approximate uniqueness check algorithms `firstBitsPlusStrides`, `theFirstKPercent`, `ModifiedMuellerHash`.
5. `negType` is an integer in {`1`, `2`} for choosing one of the strategies for using negation `negOfPhi` or `negOfChar`.

For example, to run the first example from the existing benchmarks, use the code below.

In [ ]:
input_path = "/content/ltl-learning-on-gpus/benchmarks/standard_benchmarks_1/flie_benchmarks/TracesFiles/abscence/Ab0.trace"

c1 = 1              # cost of (a) alphabet
c2 = 1              # cost of (~) negation
c3 = 1              # cost of (&) intersection
c4 = 1              # cost of (|) union
c5 = 1              # cost of (X) neXt
c6 = 1              # cost of (F) Finally
c7 = 1              # cost of (G) Globally
c8 = 1              # cost of (U) Until
maxCost = 500       # A big-enough cost
RlxUnqChkType = 3   # 3 = ModifiedMuellerHash
NegType = 2          # 2 = negOfChar

# Run the code
! ./ltli6463 {input_path} {c1} {c2} {c3} {c4} {c5} {c6} {c7} {c8} {maxCost} {RlxUnqChkType} {NegType}

# <strong>Divide-and-conquer Algorithms for bigger instances</strong>

Let's import and install some additional dependencies. Please run the following script:

In [ ]:
print("Installing bitarray...")
! pip install bitarray

We are now able to call the GPU code multiple times for larger instances in some divide-and-conquer algorithms and combine the results using higher-level scripts written in Python. Apart from handling timeouts, we now have two important additional arguments:

- `dcAlgo`: Specifies the divide-and-conquer algorithm to use, which can be either `DetSplit` or `RandSplit`.
- `window`: Indicates the maximum number of traces to pass to the GPU code. In this version, `window` <= 64.

Please run the script below, which is another example of the existing benchmarks:

In [ ]:
import os
import sys
sys.path.append("/content/ltl-learning-on-gpus/code")
from dc import *

# Arguments:
input_path = "/content/ltl-learning-on-gpus/benchmarks/standard_benchmarks_1/flie_benchmarks/TracesFiles/existence/E7.trace"
dcAlgo = RandSplit                  # Random splitting
window = 64                         # 64 is max traces number in this version
costfun = [1, 1, 1, 1, 1, 1, 1, 1]  # Costs of (a, ~, &, |, X, F, G, U)
maxCost = 500                       # A big-enough cost
RlxUnqChkType = 3                   # 3 = ModifiedMuellerHash
NegType = 2                         # 2 = negOfChar
timeout = 2000                      # 2000 sec

# Run the code
runDC(input_path, dcAlgo, window, costfun, maxCost, RlxUnqChkType, NegType, timeout)

# <strong>More Info</strong>

If you need further information, please refer to our paper

https://arxiv.org/abs/2402.12373

or feel free to contact the authors directly.